In [17]:
# import numpy as np
# import pandas as pd
# from sklearn.metrics.pairwise import euclidean_distances
# from sklearn.metrics.pairwise import linear_kernel
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.feature_extraction.text import TfidfVectorizer
# from collections import Counter
# import mysql.connector
# import warnings
# warnings.filterwarnings("ignore")



# # join and concatenate inputted columns
# def create_soup(x, features):
#     soup_parts = [str(x[feature]) for feature in features if x[feature] is not None]  # Convert to string and filter out None values
#     return ' '.join(soup_parts)

# # count number of likeable elements for film
# def count_likeable(row):
#     features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     atts = sum(1 for col in row[features] if col is not None)
#     cast = len(row['cast'].split(','))
#     return atts+cast

# # get user loved films from db
# def get_loved_films(user_id):

#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT tconst FROM user_loved_films WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     rows = mycursor.fetchall()

#     tconst_list = [tconst[0] for tconst in rows]

#     mycursor.close()
#     mydb.close()

#     loved_films_df = data[data['tconst'].isin(tconst_list)]

#     return loved_films_df

# # get user liked attributes from db
# def get_liked_attributes(user_id):

#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_liked_attributes WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     attribute_fetch = mycursor.fetchall()

#     tconst_list = [row[1] for row in attribute_fetch]
#     attribute_bin = [row[1:] for row in attribute_fetch]

#     mycursor.close()
#     mydb.close()

#     attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     liked_attributes_df = pd.DataFrame(columns=attributes_template)

#     x = 0
#     for tconst, att_bin in zip(tconst_list, attribute_bin):
#         row_values = [tconst]  # Initialize row values with tconst
#         for liked, attribute in zip(att_bin[1:], attributes_template[1:]):
#             film_att = data[data['tconst'] == tconst]

#             if liked == 1:
#                 row_values.append(film_att[attribute].values[0])
#             else:
#                 row_values.append(None) 
#         liked_attributes_df.loc[x] = row_values
#         x+=1

#     return liked_attributes_df

# # get user liked cast from db
# def get_liked_cast(user_id):
#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_liked_cast WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     cast_fetch = mycursor.fetchall()

#     mycursor.close()
#     mydb.close()

#     cast = [row[1:] for row in cast_fetch]
#     liked_cast_df = pd.DataFrame(cast, columns=['tconst', 'name'])

#     return liked_cast_df

# # get user watchlist from db
# def get_watchlist(user_id):
#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_watchlist WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))


#     watchlist_fetch = mycursor.fetchall()

#     mycursor.close()
#     mydb.close()

#     tconst_list = [row[1] for row in watchlist_fetch]
#     attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     watchlist_df = pd.DataFrame(columns=attributes_template)

#     x = 0
#     for tconst in tconst_list:
#         watchlist_df.loc[x] = data[data['tconst'] == tconst][attributes_template].values[0]
#         x+=1

#     return watchlist_df

# # calculate % of liked attributes of film in profile
# def calculate_percentage(row):
#     total_attributes = row['total_likeable']
#     total_liked = row['num_liked_atts']
#     percentage = total_liked / total_attributes
#     return percentage 

# # count liked attributes for film
# def count_liked(row):
#     features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     atts = sum(1 for col in row[features] if pd.notna(col))  # Change this line
#     cast = 0
#     if not pd.isna(row['cast']):
#         cast = len(row['cast'].split(','))
    
#     return atts + cast

# # CREATE USER PROFILE
# def get_user_profile(user_id):
#     # get user film preferences from db
#     lovedFilms = get_loved_films(user_id)
#     likedAtt = get_liked_attributes(user_id)
#     likedCast = get_liked_cast(user_id)
#     watchlist = get_watchlist(user_id)

#     # merge liked cast with liked attribute table
#     likedCast_grouped = likedCast.groupby('tconst')['name'].apply(lambda x: ', '.join(x.dropna())).reset_index()
#     likedAttributes = pd.merge(likedAtt, likedCast_grouped, on='tconst', how='outer') #outer join instead of left?
#     likedAttributes.rename(columns={'name': 'cast'}, inplace=True)

#     # format user_profile dataframe
#     merged_love_liked = pd.concat([lovedFilms, likedAttributes], ignore_index=True)
#     user_profile = pd.merge(merged_love_liked, data[['tconst', 'total_likeable']], on='tconst', how='left')
#     user_profile = user_profile.drop(columns=['poster', 'total_likeable_x'])
#     user_profile.rename(columns={'total_likeable_y': 'total_likeable'}, inplace=True)

#     # initialise columns to calculate likeage
#     user_profile['num_liked_atts'] = 0
#     user_profile['likeage'] = 0.0
#     # count total liked attributes for each film in profile
#     for index, film in user_profile.iterrows():
#         tconst = film['tconst']
#         user_profile.loc[user_profile['tconst'] == tconst, 'num_liked_atts'] = count_liked(film)

#     # calculate likeage for each film in profile
#     user_profile['likeage'] = user_profile.apply(lambda row: calculate_percentage(row), axis=1)
#     user_profile.drop(columns=['num_liked_atts', 'total_likeable'], inplace=True)

#     return (user_profile, lovedFilms)

# # split user profile into likes by group
# def collate_liked_groups(user_profile):
#     group_dataframes = []

#     # Define dictionary to map group names to corresponding columns
#     group_columns = {
#         'liked_title_plot': ['primaryTitle', 'plot', 'likeage'],
#         'liked_cast': ['cast', 'likeage'],
#         'liked_crew': ['director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'likeage'],
#         'liked_genre': ['genres', 'likeage'],
#         'liked_meta': ['averageRating', 'startYear', 'runtimeMinutes', 'likeage']
#     }

#     # Iterate over the dictionary and create group dataframes
#     for group_name, columns in group_columns.items():
#         group_df = user_profile[columns].copy()
#         # Drop rows with all NaN values in specified columns
#         group_df = group_df.dropna(subset=columns[:-1], how='all')
#         group_dataframes.append(group_df)

#     return group_dataframes

# # cosine similarity vector with tf-idf between films in row and column
# def create_similarity_vector(row, column):
#     tfidf = TfidfVectorizer(stop_words='english')
#     row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
#     row_soup = row_soup_temp.fillna('')
#     row_matrix = tfidf.fit_transform(row_soup)

#     column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
#     column_soup = column_soup_temp.fillna('')
#     column_matrix = tfidf.transform(column_soup)

#     return linear_kernel(row_matrix, column_matrix)

# # euclidean distance vector for numerical attributes of film
# def create_euclidean_vector(row, column):
#     scaler = MinMaxScaler()
#     row = row.fillna('')
#     column = column.fillna(0)

#     row_normalized = scaler.fit_transform(row)
#     column_normalized = scaler.transform(column)
#     distances = euclidean_distances(row_normalized, column_normalized)
#     euclidean_matrix = 1 / (1 + distances)
#     return euclidean_matrix

# # get top N films similar to user profile based on input vector (NOT CURRENTLY IN USE)
# def get_similar_films(vector, N):
#     filtered_vector = vector[~data['tconst'].isin(lovedFilms['tconst'])] #don't recommend film the user has already loved
#     mean_similarity = np.mean(filtered_vector, axis=1)
#     sorted_indices = np.argsort(mean_similarity)[::-1]
#     top_N = sorted_indices[:N]
#     return data.iloc[top_N]

# # function to calculate unified recommendations
# def get_unified_recommendations(user_profile_groups, similarity_vectors, N):

#     weighted_scores = {}
    
#     # scale similarity score in respective vector based on likeage of feature
#     for group, attributes in user_profile_groups.items():
#         similarity_vector = similarity_vectors[group][~data['tconst'].isin(user_profile['tconst'])] #don't recommend films already in profile
#         likeage_array = np.array(list(attributes['likeage'].tolist()))
#         weighted_similarity = similarity_vector * likeage_array

#         # meta columns have higher similarity since its numerical so adjust for this
#         if group == 'meta':
#             scaling_factor = 1 - weighted_similarity
#             weighted_similarity *= scaling_factor

#         # dictionary of groups and weighted similarity vectors 
#         weighted_scores[group] = weighted_similarity

#      # combine weighted similarity scores across all groups
#     combined_scores = np.sum(list(weighted_scores.values()), axis=0)

#     # calculate mean similarity scores
#     mean_similarity = np.mean(combined_scores, axis=1)

#     # sort the mean similarity scores and retrieve the top N indices
#     sorted_indices = np.argsort(mean_similarity)[::-1][:N]

#     return data.iloc[sorted_indices]


# data = pd.read_json('webpage/films.json')
# data['total_likeable'] = data.apply(lambda x: count_likeable(x), axis=1)
# data.head()


# # get update user profile and loved films
# get_profile = get_user_profile('57')
# user_profile = get_profile[0]
# lovedFilms = get_profile[1]

# # get groups of liked attributes
# grouped_likes = collate_liked_groups(user_profile)
# liked_plot = grouped_likes[0]
# liked_cast = grouped_likes[1]
# liked_crew = grouped_likes[2]
# liked_genre = grouped_likes[3]
# liked_meta = grouped_likes[4]

# # Define the list of features excluding 'likeage'
# plot_features = [col for col in liked_plot.columns if col != 'likeage']
# crew_features = [col for col in liked_crew.columns if col != 'likeage']
# cast_features = [col for col in liked_cast.columns if col != 'likeage']
# genre_features = [col for col in liked_genre.columns if col != 'likeage']
# meta_features = [col for col in liked_meta.columns if col != 'likeage']

# # Call the function with the selected columns
# plot_matrix = create_similarity_vector(data[plot_features], liked_plot[plot_features])
# crew_matrix = create_similarity_vector(data[crew_features], liked_crew[crew_features])
# cast_matrix = create_similarity_vector(data[cast_features], liked_cast[cast_features])
# genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])
# meta_matrix = create_euclidean_vector(data[meta_features], liked_meta[meta_features])


# # Similarity vectors for each group
# similarity_vectors = {
#     'plot': plot_matrix,
#     'cast': cast_matrix,
#     'crew': crew_matrix,
#     'genre': genre_matrix,
#     'meta': meta_matrix
# }

# user_profile_groups = {
#     'plot': liked_plot,
#     'cast': liked_cast,
#     'crew': liked_crew,
#     'genre': liked_genre,
#     'meta': liked_meta
# }


# get_unified_recommendations(user_profile_groups, similarity_vectors, 20)



In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import mysql.connector
import warnings
warnings.filterwarnings("ignore")

# data = pd.read_json('webpage/films.json')

# join and concatenate inputted columns
def create_soup(x, features):
    soup_parts = [str(x[feature]) for feature in features if x[feature] is not None]  # Convert to string and filter out None values
    return ' '.join(soup_parts)

# count number of likeable elements for film
def count_likeable(row):
    features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    atts = sum(1 for col in row[features] if col is not None)
    cast = len(row['cast'].split(','))
    return atts+cast

# get user loved films from db
def get_loved_films(user_id):

    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT tconst FROM user_loved_films WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    rows = mycursor.fetchall()

    tconst_list = [tconst[0] for tconst in rows]

    mycursor.close()
    mydb.close()

    loved_films_df = data[data['tconst'].isin(tconst_list)]

    return loved_films_df

# get user liked attributes from db
def get_liked_attributes(user_id):

    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_liked_attributes WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    attribute_fetch = mycursor.fetchall()

    tconst_list = [row[1] for row in attribute_fetch]
    attribute_bin = [row[1:] for row in attribute_fetch]

    mycursor.close()
    mydb.close()

    attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    liked_attributes_df = pd.DataFrame(columns=attributes_template)

    x = 0
    for tconst, att_bin in zip(tconst_list, attribute_bin):
        row_values = [tconst]  # Initialize row values with tconst
        for liked, attribute in zip(att_bin[1:], attributes_template[1:]):
            film_att = data[data['tconst'] == tconst]

            if liked == 1:
                row_values.append(film_att[attribute].values[0])
            else:
                row_values.append(None) 
        liked_attributes_df.loc[x] = row_values
        x+=1

    return liked_attributes_df

# get user liked cast from db
def get_liked_cast(user_id):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_liked_cast WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    cast_fetch = mycursor.fetchall()

    mycursor.close()
    mydb.close()

    cast = [row[1:] for row in cast_fetch]
    liked_cast_df = pd.DataFrame(cast, columns=['tconst', 'name'])

    return liked_cast_df

# get user watchlist from db
def get_watchlist(user_id):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_watchlist WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))


    watchlist_fetch = mycursor.fetchall()

    mycursor.close()
    mydb.close()

    tconst_list = [row[1] for row in watchlist_fetch]
    attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    watchlist_df = pd.DataFrame(columns=attributes_template)

    x = 0
    for tconst in tconst_list:
        watchlist_df.loc[x] = data[data['tconst'] == tconst][attributes_template].values[0]
        x+=1

    return watchlist_df

# calculate % of liked attributes of film in profile
def calculate_percentage(row):
    total_attributes = row['total_likeable']
    total_liked = row['num_liked_atts']
    percentage = total_liked / total_attributes
    return percentage 

# count liked attributes for film
def count_liked(row):
    features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    atts = sum(1 for col in row[features] if pd.notna(col))  # Change this line
    cast = 0
    if not pd.isna(row['cast']):
        cast = len(row['cast'].split(','))
    
    return atts + cast

# CREATE USER PROFILE
def get_user_profile(user_id):
    # get user film preferences from db
    lovedFilms = get_loved_films(user_id)
    likedAtt = get_liked_attributes(user_id)
    likedCast = get_liked_cast(user_id)
    watchlist = get_watchlist(user_id)

    # merge liked cast with liked attribute table
    likedCast_grouped = likedCast.groupby('tconst')['name'].apply(lambda x: ', '.join(x.dropna())).reset_index()
    likedAttributes = pd.merge(likedAtt, likedCast_grouped, on='tconst', how='outer') #outer join instead of left?
    likedAttributes.rename(columns={'name': 'cast'}, inplace=True)

    # format user_profile dataframe
    merged_love_liked = pd.concat([lovedFilms, likedAttributes], ignore_index=True)
    user_profile = pd.merge(merged_love_liked, data[['tconst', 'total_likeable']], on='tconst', how='left')
    user_profile = user_profile.drop(columns=['poster', 'total_likeable_x'])
    user_profile.rename(columns={'total_likeable_y': 'total_likeable'}, inplace=True)

    # initialise columns to calculate likeage
    user_profile['num_liked_atts'] = 0
    user_profile['likeage'] = 0.0
    # count total liked attributes for each film in profile
    for index, film in user_profile.iterrows():
        tconst = film['tconst']
        user_profile.loc[user_profile['tconst'] == tconst, 'num_liked_atts'] = count_liked(film)

    # calculate likeage for each film in profile
    user_profile['likeage'] = user_profile.apply(lambda row: calculate_percentage(row), axis=1)
    user_profile.drop(columns=['num_liked_atts', 'total_likeable'], inplace=True)

    return (user_profile, lovedFilms)

# get array of names from inputted dataframe
def extract_names(data):
    names = set()
    for column in data.columns:
        for value in data[column]:
            if value and isinstance(value, str) and value.lower() not in ['none', 'null', 'nan']:
                names.update(value.split(", "))
    return list(names)

# split user profile into likes by group
def collate_liked_groups(user_profile):
    group_dataframes = []

    # Define dictionary to map group names to corresponding columns
    group_columns = {
        'liked_title_plot': ['primaryTitle', 'plot', 'likeage'],
        'liked_cast': ['cast', 'likeage'],
        'liked_crew': ['director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'likeage'],
        'liked_genre': ['genres', 'likeage'],
        'liked_meta': ['averageRating', 'startYear', 'runtimeMinutes', 'likeage']
    }

    # Iterate over the dictionary and create group dataframes
    for group_name, columns in group_columns.items():
        group_df = user_profile[columns].copy()
        # group_df = group_df.dropna(subset=columns[:-1], how='all')
        group_dataframes.append(group_df)

    return group_dataframes

# cosine similarity vector with tf-idf between films in row and column
def create_similarity_vector(row, column):
    tfidf = TfidfVectorizer(stop_words='english')
    row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
    row_soup = row_soup_temp.fillna('')
    row_matrix = tfidf.fit_transform(row_soup)

    column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
    column_soup = column_soup_temp.fillna('')
    column_matrix = tfidf.transform(column_soup)

    return linear_kernel(row_matrix, column_matrix)

# euclidean distance vector for numerical attributes of film
def create_euclidean_vector(row, column):
    scaler = MinMaxScaler()
    row = row.fillna('')
    column = column.fillna(0)

    row_normalized = scaler.fit_transform(row)
    column_normalized = scaler.transform(column)
    distances = euclidean_distances(row_normalized, column_normalized)
    euclidean_matrix = 1 / (1 + distances)
    return euclidean_matrix

# get top N films similar to user profile based on input vector (NOT CURRENTLY IN USE)
def get_similar_films(vector, exclude):
    mean_similarity = np.mean(vector, axis=1)
    sorted_indices = np.argsort(mean_similarity)[::-1]
    top_N = sorted_indices[:]
    sorted_films = data.iloc[top_N]
    filtered_recommendations = sorted_films[~sorted_films['tconst'].isin(exclude['tconst'])]

    return filtered_recommendations

# function to calculate combined recommendations
def get_combined_recommendations(user_profile_groups, similarity_vectors, exclude_films):

    weighted_scores = {}
    
    # scale similarity score in respective vector based on likeage of feature
    for group, attributes in user_profile_groups.items():
        similarity_vector = similarity_vectors[group]
        likeage_array = np.array(list(attributes['likeage'].tolist()))
        weighted_similarity = similarity_vector * likeage_array

        # dictionary of groups and weighted similarity vectors 
        weighted_scores[group] = weighted_similarity

     # combine weighted similarity scores across all groups
    combined_scores = np.sum(list(weighted_scores.values()), axis=0)

    # calculate mean similarity scores
    mean_similarity = np.mean(combined_scores, axis=1)

    # sort the mean similarity scores and retrieve the top N indices
    sorted_indices = np.argsort(mean_similarity)[::-1]

    sorted_films = data.iloc[sorted_indices]

    sorted_films['similarity'] = mean_similarity[sorted_indices] / 5

    filtered_recommendations = sorted_films[~sorted_films['tconst'].isin(exclude_films['tconst'])]

    return filtered_recommendations


# data['total_likeable'] = data.apply(lambda x: count_likeable(x), axis=1)


In [22]:
# load whole films dataset from db
def loadAllFilms():

    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM all_films"

    mycursor.execute(sql_query)
    
    columns = [col[0] for col in mycursor.description]

    films = mycursor.fetchall()

    mycursor.close()
    mydb.close()

    films_data = pd.DataFrame(films, columns=columns)

    return films_data

def get_watchlist(user_id, allFilms):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_watchlist WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))


    watchlist_fetch = mycursor.fetchall()

   

    mycursor.close()
    mydb.close()

    tconst_list = [row[1] for row in watchlist_fetch]

    attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'poster']
    watchlist_df = pd.DataFrame(columns=attributes_template)
    x = 0
    for tconst in tconst_list:
        # Check if any films match the given tconst
        films_matching_tconst = allFilms[allFilms['tconst'] == tconst]
        if not films_matching_tconst.empty:
            # If there are matching films, add them to the watchlist_df
            watchlist_df.loc[x] = films_matching_tconst[attributes_template].values[0]
            x += 1
        else:
            # Handle the case when no films match the tconst (optional)
            print(f"No films found for tconst: {tconst}")

    return watchlist_df

data = loadAllFilms()

watchlist = get_watchlist('57', data)


watchlist

No films found for tconst: tt0069966
No films found for tconst: tt0071864
No films found for tconst: tt0075164
No films found for tconst: tt0086170
No films found for tconst: tt0099520
No films found for tconst: tt0106418
No films found for tconst: tt0111127
No films found for tconst: tt0121153
No films found for tconst: tt0121536
No films found for tconst: tt0128371
No films found for tconst: tt0164835
No films found for tconst: tt0172089
No films found for tconst: tt0202300
No films found for tconst: tt0269856
No films found for tconst: tt0291394
No films found for tconst: tt0312320
No films found for tconst: tt0347808
No films found for tconst: tt0375529
No films found for tconst: tt0765447
No films found for tconst: tt0799949
No films found for tconst: tt10243678
No films found for tconst: tt10639214
No films found for tconst: tt10833732
No films found for tconst: tt1086841
No films found for tconst: tt12445782
No films found for tconst: tt14039346
No films found for tconst: tt1529

,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster
0,tt0049434,The Last Wagon,When a handful of settlers survive an Apache a...,7.0,"Drama,Western",99,1956,"George Mathews, Nick Adams, Felicia Farr, ...",Delmer Daves,Wilfrid M. Cline,"James Edward Grant, Delmer Daves, Gwen Bagni",William B. Hawks,Hugh S. Fowler,Lionel Newman,/2FHbwL1RZCeAsBRSBVuxYRj9Mhl.jpg
1,tt0051378,Elevator to the Gallows,A self-assured businessman murders his employe...,7.9,"Crime,Drama,Thriller",91,1958,"Maurice Ronet, Micheline Bona, Jacqueline St...",Louis Malle,Henri Decaë,"Roger Nimier, Louis Malle, Noël Calef",Jean Thuillier,Léonide Azar,Miles Davis,/dLbf8LtbLQFM7KUaVgwkfKIPqXu.jpg
2,tt0058994,Bruce's Fingers,After malicious gangsters capture Bruce Wong's...,4.9,"Action,Crime,Drama",91,1976,"Tao Chiang, Lieh Lo, Nora Miao, Rose Cheng...",Joseph Velasco,Hak-Wai Ho,"Joseph Velasco, Yun-Ping Wu","Liu En-Ze, Joseph Velasco",Nan Hsiao,Lawrence Chan,/iJF8yMqI14LndLwGpfWiOuoCsvu.jpg
3,tt0060899,Ride Beyond Vengeance,Jonas Trapp falls in love with the beautiful J...,6.2,"Drama,Western",101,1966,"Paul Fix, Gary Merrill, Chuck Connors, Clau...",Bernard McEveety,Lester Shorr,"Andrew J. Fenady, Al Dewlen",Andrew J. Fenady,Otho Lovering,Richard Markowitz,/qO3XemtoN9e7AjSW2kA9EkZ7Mf7.jpg
4,tt0064100,Bob & Carol & Ted & Alice,After returning to Los Angeles from a group th...,6.7,"Comedy,Drama,Romance",105,1969,"Elliott Gould, Robert Culp, Dyan Cannon, Ce...",Paul Mazursky,Charles Lang,"Paul Mazursky, Larry Tucker",Larry Tucker,Stuart H. Pappé,Quincy Jones,/8fDLuB1q3JycYJPK0F65OYO3lAX.jpg
5,tt0078350,The Swarm,Scientist Dr. Bradford Crane and army general ...,4.5,"Horror,Thriller",116,1978,"Olivia de Havilland, Michael Caine, Patty Du...",Irwin Allen,Fred J. Koenekamp,"Arthur Herzog III, Stirling Silliphant",Irwin Allen,Harold F. Kress,Jerry Goldsmith,/uAuPFiFBmqcyTcJOBMLH6z70f1y.jpg
6,tt0090290,The War Boy,The 12-year-old son of a freedom fighter grows...,4.8,Drama,86,1985,"Vjenceslav Kapural, Helen Shaver, Kenneth We...",Allan Eastman,"Fred Luchetti, Tomislav Pinter","Julius Kohanyi, Jim Osborne","Jason Hopley, William T. Marshall",Vesna Kreber,Peter Wetzler,/ftixNMRFHNPJogpTi1rhDIUua5K.jpg
7,tt0093456,Lust for Freedom,A former female cop is framed by corrupt polic...,3.8,"Crime,Drama,Thriller",92,1987,"Judi Trevor, John Tallman, Howard Knight, ...",Eric Louzil,Ron Chapman,"Craig Kusaba, Duke Howard, Eric Louzil",Eric Louzil,"David Khachatorian, Steve Mann, Thomas R. Rond...",John Massari,/dK0hf7KjKpf5WyVyfjhwpAZwpOz.jpg
8,tt0096427,We Think the World of You,"An aimless young man, Johnny, is sent prison. ...",5.9,Comedy,90,1988,"Kerry Wise, Frances Barber, Liz Smith, She...",Colin Gregg,Michael Garfath,"J.R. Ackerley, Hugh Stoddart",Tommaso Jandelli,Peter Delfgou,Julian Jacobson,/kwyuYjPd0NNiQbsRRbJ0JqPZw5Y.jpg
9,tt0097136,Cutting Class,High school student Paula Carson's affections ...,4.5,"Comedy,Crime,Horror",91,1989,"Eric Boles, Dirk Blocker, Robert Glaudini, ...",Rospo Pallenberg,Avraham Karpick,Steve Slavkin,"Donald R. Beck, Rudy Cohen","Bill Butler, Natan Zahavi",Jill Fraser,/o0PYYaN9JRrm66C42HhjdxwxNVM.jpg


In [7]:
import re #regular expressions

attributes = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes','cast' ,'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
data['soup'] = data.apply(lambda x: create_soup(x, attributes), axis=1)

# def search_films(keywords):
#      # Convert keywords to lowercase for case-insensitive matching
#     keywords_lower = set(keyword.lower() for keyword in keywords)

#     filtered_films = data.copy()

#     for keyword in keywords_lower:
#     # Filter films that contain the keyword in the soup
#         filtered_films = filtered_films[filtered_films['soup'].str.lower().str.contains(keyword)]

#     return filtered_films
def search_films(sentences):
    # Initialize an empty DataFrame to store the filtered films
    filtered_films = pd.DataFrame()

    # Iterate over each sentence in the array
    for sentence in sentences:
        # Tokenize the sentence into individual keywords
        keywords = sentence.split()

        # Convert keywords to lowercase for case-insensitive matching
        keywords_lower = set(keyword.lower() for keyword in keywords)

        # Filter films that contain all keywords in the soup
        for keyword in keywords_lower:
            # Filter films that contain the current keyword
            films_with_keyword = data[data['soup'].str.lower().str.contains(keyword)]
            
            # Update the filtered films DataFrame with films containing the current keyword
            filtered_films = pd.concat([filtered_films, films_with_keyword])

        # Drop duplicates from the filtered DataFrame
        filtered_films = filtered_films.drop_duplicates()

    return filtered_films


# Example usage:
filters = ['tom cruise action', 'comedy']
pd.DataFrame(search_films(filters))

# pd.DataFrame(search_films_by_keywords(film_soup, filters))



,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable,soup
882,tt0175880,Magnolia,An epic mosaic of many interrelated characters...,8.0,Drama,188,1999,"Tom Cruise, Jason Robards, Philip Seymour Hoff...",Paul Thomas Anderson,Robert Elswit,None,JoAnne Sellar,Dylan Tichenor,Jon Brion,/uq2u8HgtLFJkjNq2kHb2jvipIPT.jpg,15,Magnolia An epic mosaic of many interrelated c...
1789,tt1275848,Twilight Syndrome: Dead Cruise,In this J-horror flick based on a popular vide...,4.0,Horror,78,2008,"Naoki Nokubo, Megumi Seki, Saki Terashima, Meg...",Takeshi Furusawa,Shin Hayasaka,Kaworu Asakusa,"Issei Shibata, Yoshihiro Yagihara",None,Taku Unami,/Alr0MUsSQotKbbNVhkXig0I8PDX.jpg,15,Twilight Syndrome: Dead Cruise In this J-horro...
1792,tt0062759,The Diamond Arm,"In the southern town, a gang of ""currency trad...",8.3,"Adventure,Comedy,Crime",94,1969,"Yuriy Nikulin, Andrey Mironov, Anatoliy Papano...",Leonid Gaidai,Igor Chernykh,"Moris Slobodskoy, Yakov Kostyukovskiy",None,Valentina Yankovskaya,Aleksandr Zatsepin,/iwVRDsZAuJmulnPKeYxhezzpr9A.jpg,15,"The Diamond Arm In the southern town, a gang o..."
2407,tt0054310,Sink the Bismarck!,The story of the breakout of the German battle...,7.1,"Action,Drama,History",97,1960,"Kenneth More, Carl Möhner, Laurence Naismith, ...",Lewis Gilbert,Christopher Challis,"Edmund H. North, C.S. Forester",John Brabourne,None,Clifton Parker,/dMGCXCAPA9zJnzwdrIPO0TkSt2x.jpg,15,Sink the Bismarck! The story of the breakout o...
2902,tt0411096,All My Life,Sam and Louie meet aboard the Star Cruise. Des...,6.9,"Drama,Romance",110,2004,"Aga Muhlach, Ricky Davao, Kristine Hermosa, Ce...",Laurenti M. Dyogi,Martin Jimenez,"Mia Concio, Mel Mendoza-Del Rosario",Elma S. Medua,None,Raul Mitra,/58l4QZ2yykbZDuSlQNg2HI2wQQz.jpg,15,All My Life Sam and Louie meet aboard the Star...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23616,tt12143014,Amalgama,"Four dentists, three men and a woman, meet at ...",4.8,"Comedy,Drama",92,2020,"Manolo Cardona, Miguel Rodarte, Tony Dalton, S...",Carlos Cuarón,Alfredo Altamirano,Luis Usabiaga,"Juancho Cardona, Alex Garcia",None,Felipe Perez Santiago,/uoBXUWujQdWtH3a5jwtIcn9z9vb.jpg,15,"Amalgama Four dentists, three men and a woman,..."
23625,tt0111074,The Stolen Joy,The residents of the Mokattam area are in a st...,6.3,Comedy,120,1994,"Maged El-Masri, Hassan Hosny, Lucy, Hanan Turk",Daoud Abdel Sayed,Tarek El-Telmisany,Khairi Shalabi,None,Ahmad Metwalli,Rajeh Dawood,/qhc44NPbASdOXayFNrjd3taBC95.jpg,15,The Stolen Joy The residents of the Mokattam a...
23628,tt11880264,Lantern's Lane,A recent college graduate and her estranged hi...,3.1,"Comedy,Horror,Thriller",95,2021,"Skip Howland, Brooke Butler, Sydney Carvill, A...",Justin LaReau,Tate McCurdy,None,Lydia Cedrone,Ziyun Chen,Keith Sterling,/4xJXC8tRAUKZeS7YoFK9UvEx8hb.jpg,15,Lantern's Lane A recent college graduate and h...
23633,tt15422224,Slotherhouse,"Emily Young, a senior, wants to be elected as ...",4.7,"Comedy,Horror,Thriller",93,2023,"Andrew Horton, Lisa Ambalavanar, Sydney Craven...",Matthew Goodhue,None,"Bradley Fowler, Cady Lanigan",Mark David,Mike Mendez,Sam Ewing,/oMsOOdI43XC06vr4uCqpJ2hyfd7.jpg,15,"Slotherhouse Emily Young, a senior, wants to b..."


In [120]:
# get update user profile and loved films
get_profile = get_user_profile('57')
user_profile = get_profile[0]
lovedFilms = get_profile[1]

# get groups of liked attributes
grouped_likes = collate_liked_groups(user_profile)
liked_plot = grouped_likes[0]
liked_cast = grouped_likes[1]
liked_crew = grouped_likes[2]
liked_genre = grouped_likes[3]
liked_meta = grouped_likes[4]

# Define the list of features excluding 'likeage'
plot_features = [col for col in liked_plot.columns if col != 'likeage']
crew_features = [col for col in liked_crew.columns if col != 'likeage']
cast_features = [col for col in liked_cast.columns if col != 'likeage']
genre_features = [col for col in liked_genre.columns if col != 'likeage']
meta_features = [col for col in liked_meta.columns if col != 'likeage']

# Call the function with the selected columns
plot_matrix = create_similarity_vector(data[plot_features], liked_plot[plot_features])
crew_matrix = create_similarity_vector(data[crew_features], liked_crew[crew_features])
cast_matrix = create_similarity_vector(data[cast_features], liked_cast[cast_features])
genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])
meta_matrix = create_euclidean_vector(data[meta_features], liked_meta[meta_features])

# Similarity vectors for each group
similarity_vectors = {
    'plot': plot_matrix,
    'cast': cast_matrix,
    'crew': crew_matrix,
    'genre': genre_matrix,
    'meta': meta_matrix
}

# meta_matrix
user_profile_groups = {
    'plot': liked_plot,
    'cast': liked_cast,
    'crew': liked_crew,
    'genre': liked_genre,
    'meta': liked_meta
}

user_profile
# recommended = get_combined_recommendations(user_profile_groups, similarity_vectors, lovedFilms)
# get_similar_films(genre_matrix, lovedFilms)[:20]


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,likeage
0,tt0367027,Shortbus,"In post-9/11 New York City, an eclectic group ...",6.4,"Comedy,Drama,Romance",101,2006,"Peter Stickles, PJ DeBoy, Paul Dawson, Sook-Yi...",John Cameron Mitchell,Frank G. DeMarco,None,"Howard Gertler, Tim Perell",Brian A. Kates,Yo La Tengo,1.000000
1,tt0120689,The Green Mile,A supernatural tale set on death row in a Sout...,8.6,"Crime,Drama,Fantasy",189,1999,"Tom Hanks, Michael Clarke Duncan, David Morse,...",Frank Darabont,David Tattersall,Stephen King,David Valdes,Richard Francis-Bruce,Thomas Newman,1.000000
2,tt0483835,Four Windows,"They want to get away from each other, but the...",4.6,Drama,80,2006,"Thorsten Merten, Frank Droese, Margarita Broic...",Christian Moris Müller,Jürgen Jürges,None,Philipp Budweg,Maja Stieghorst,Chandra Fleig,1.000000
3,tt0057043,The Happy Musketeers,The Big Bad Wolf and the Stinky Little Skunk f...,5.1,"Adventure,Comedy,Drama",86,1961,"Elmo Michel, Santanón, Ariadne Welter, Marina ...",Roberto Rodríguez,Alex Phillips,Manuel R. Ojeda,Aurelio García Yévenes,José W. Bustos,Sergio Guerrero,1.000000
4,tt0373921,Mother of the Bride,This magnificently madcap comedy classic stars...,7.5,"Comedy,Drama,Family",102,1963,"Imad Hamdi, Hasan Youssef, Samira Ahmed, Madih...",Atef Salem,Masud Isa,"Abdel Hai Adib, Abdel Hamid Gouda",Naguib Khoury,Hussein Ahmed,None,1.000000
5,tt0178576,L'hiver bleu,L’Hiver bleu is an Abitibian chronicle that pr...,5.7,Drama,82,1979,"Michel Chenier, Réjean Roy, Christiane Lévesqu...",André Blanchard,Alain Dupras,Jeanne-Mance Delisle,Marguerite Duparc,None,"Abbittibbi, Les Anonymes d'Évias",1.000000
6,tt1512235,Super,After his wife falls under the influence of a ...,6.7,"Action,Comedy,Crime",96,2010,"Rainn Wilson, Elliot Page, Kevin Bacon, Liv Tyler",James Gunn,Steve Gainer,None,"Miranda Bailey, Ted Hope",Cara Silverman,Tyler Bates,1.000000
7,tt7454396,A Mother,"After getting released from prison, Eeva retur...",5.8,Drama,91,2019,"Matti Pajulahti, Jaana Saarinen, Helmi-Leena N...",Samppa Batal,None,Krista Hannula,"Maarit Laaksoharju, Miikka Pakarinen",Tuomas Kohtamäki,Antti L.S. Ikonen,1.000000
8,tt0049103,None,None,NaN,"Horror,Sci-Fi",None,None,NaN,None,None,None,None,None,None,0.066667
9,tt0083067,None,None,NaN,"Comedy,Musical",None,None,NaN,None,None,None,None,None,None,0.066667


In [122]:
from sklearn.feature_extraction.text import CountVectorizer


# cosine similarity vector with tf-idf between films in row and column
def create_soup(x, features):
    soup_parts = [1 if x[feature] else 0 for feature in features]  # Encode genre presence as binary values
    return soup_parts

# join and concatenate inputted columns
def create_soup(x, features):
    soup_parts = [str(x[feature]) for feature in features if x[feature] is not None]  # Convert to string and filter out None values
    return ' '.join(soup_parts)


def create_similarity_vector(row, column):
    tfidf = CountVectorizer(stop_words='english')
    tfidf = TfidfVectorizer(stop_words='english')

    row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
    row_soup = row_soup_temp.fillna('')
    row_matrix = tfidf.fit_transform(row_soup)

    column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
    column_soup = column_soup_temp.fillna('')
    column_matrix = tfidf.transform(column_soup)

    return linear_kernel(row_matrix, column_matrix)


def get_similar_films(vector):
    mean_similarity = np.mean(vector, axis=1)
    sorted_indices = np.argsort(mean_similarity)[::-1]
    top_N = sorted_indices[:]
    sorted_films = data.iloc[top_N]
    # filtered_recommendations = sorted_films[~sorted_films['tconst'].isin(exclude['tconst'])]

    return sorted_films

# genre_features = [col for col in liked_genre.columns if col != 'likeage']
# liked_genre = liked_genre.drop(columns="likeage")
x = liked_genre.apply(lambda x: create_soup(x, liked_genre.columns), axis=1)

x

genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])
get_similar_films(genre_matrix)[:25]


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable
5936,tt0104119,Dolly Dearest,An American family moves to Mexico to fabricat...,4.7,"Comedy,Drama,Horror",93,1991,"Sam Bottoms, Rip Torn, Chris Demetral, Denise ...",Maria Lease,Eric D. Andersen,"Rod Nave, Peter Sutcliffe",Daniel Cady,None,Mark Snow,/fp5o7DIp31R3T7SNzPGVKgyyTpu.jpg,15
17260,tt0096343,Unmasked Part 25,Jackson is a lonely serial killer who is reall...,5.8,"Comedy,Drama,Horror",85,1988,"Gregory Cox, Edward Brayshaw, Fiona Evans, Deb...",Anders Palm,John de Borman,"Mark Cutforth, William Shakespeare",None,Brian Peachey,Julian Wastall,/acGNEpFLYiSTArlG1utBhpqWs9f.jpg,15
13175,tt1653881,Help of God,"""Kamisama Help!"" takes place at an abandoned s...",6.0,"Comedy,Drama,Horror",87,2010,"Kazuki Katô, Kento Kaku, Takahiro Hôjô, Aimi S...",Shota Sasaki,Chigi Kanbe,Ichiro Anabuki,"Kenji Nakanishi, Hitoshi Nunokawa",None,Kuniyuki Morohashi,/4xQMwKsFq4QSjUHi8eD6Dz2lPPS.jpg,15
13169,tt14773734,Bomma Adirindi Dhimma Tirigindi,"After five years, a group of college girls pla...",6.8,"Comedy,Drama,Horror",109,2021,"Arjun Kalyan, Shakalaka Shankar, Avantika, San...",Kumar Kota,Phanindra Varma Alluri,V.S. Rao,L. Madhu,None,Bhagawath,/lA1WD2HvIsV7qA2KlSMC1Sp0EIo.jpg,15
9444,tt10441822,The Nowhere Inn,When St. Vincent sets out to make a documentar...,5.7,"Comedy,Drama,Horror",91,2020,"Ezra Buzzington, Toko Yasuda, Chris Aquilino, ...",Bill Benz,Minka Farthing-Kohl,Carrie Brownstein,"Lana Kim, Jett Steiger",Ali Greer,None,/saXbsVa6eylbaOCXpwGZVjn1fOd.jpg,15
10528,tt0432006,Puppy,"Attempting suicide, sultry but down-on-her-luc...",6.0,"Comedy,Drama,Horror",95,2005,"Bernard Curry, Terence Donovan, Nadia Townsend...",Kieran Galvin,Tom Gleeson,None,Melissa Beauford,Mark Atkin,Karel Segers,/oTm4rx4MfOIVj9Po1GAKohJXT4X.jpg,15
17414,tt0087289,Frankenstein 90,French cybernetics genius Victor Frankenstein ...,4.5,"Comedy,Drama,Horror",92,1984,"Jean Rochefort, Eddy Mitchell, Fiona Gélin, He...",Alain Jessua,William Lubtchansky,"Paul Gégauff, Mary Shelley",None,Hélène Plemiannikov,Armando Trovajoli,/cNZxL0yeDZ37YVCf05us8nbhhjF.jpg,15
510,tt0104139,Dr. Giggles,"In 1957, Evan Rendell flees after his father i...",5.3,"Comedy,Drama,Horror",95,1992,"Larry Drake, Cliff De Young, Glenn Quinn, Holl...",Manny Coto,Robert Draper,Graeme Whifler,Stuart M. Besser,Debra Neil-Fisher,Brian May,/15TOuWoaJ5C0lLcmRTQz5Lasnd.jpg,16
18101,tt5810368,Deadly Detention,Five archetypal teens serving detention find t...,3.4,"Comedy,Drama,Horror",90,2017,"Henrique Zaga, Coy Stewart, Alex Frnka, Sarah ...",Blair Hayes,Kristoffer Carrillo,"Casie Tabanou, Ali Spuck",Valentina Bove,None,Matthew Puckett,/ket8bzYI0ryOV1JhjsmWIfvs7xf.jpg,15
3787,tt14476124,Ghosty,An inspector embarks on a mission to track dow...,4.3,"Comedy,Drama,Horror",139,2023,"Madhan Babu, Yogi Babu, Kpy Bala, Kajal Aggarwal",Kalyaan,Jacob Rathinaraj,None,"Jayaram G., Sudhan Sundaram",Vijay Velukutty,Sam C.S.,/8VPfGQDBYA3fldv2C5Cxh3gQ3nR.jpg,15
